# Genera los TFRecords en base a los XMLs correspondientes a las imágenes

Adaptado de https://github.com/douglasrizzo/detection_util_scripts 

Fuente: https://medium.com/@omcar17/how-to-convert-xml-files-into-tfrecords-in-tensorflow2-0-86120b553f0b


0) Preparar el ambiente:

In [1]:
#@title Actualizar e instalar paquetes necesarios
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim
!pip install pycocotools

     |████████████████████████████████| 454.2MB 29kB/s 
     |████████████████████████████████| 4.0MB 31.4MB/s 
     |████████████████████████████████| 471kB 45.2MB/s 
     |████████████████████████████████| 1.2MB 24.5MB/s 
     |████████████████████████████████| 4.0MB 38.6MB/s 
  Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 358kB 5.2MB/s 


In [2]:
#@title Clonar el repositorio de modelos de TF si no está ya disponible
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2603, done.
remote: Counting objects: 100% (2603/2603), done.
remote: Compressing objects: 100% (2163/2163), done.
remote: Total 2603 (delta 649), reused 1201 (delta 407), pack-reused 0
Receiving objects: 100% (2603/2603), 32.56 MiB | 14.66 MiB/s, done.
Resolving deltas: 100% (649/649), done.


In [3]:
#@title Instalar el Object Detection API
# Nota: si falla por falta de requerimientos, ejecutarlo de nuevo y funcionará ;)...
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1644507 sha256=daa1039a75144a1ea39e463f1b0cba8a32371129fe7180b6a35a180bc6d5aa98
  Stored in directory: /tmp/pip-ephem-wheel-cache-vareo3ka/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.2-cp37-none-any.whl size=44011 sha256=ebcf9b8d85d408f0e75e6fb9e35ebad59edaed52506c5290a222a57fa6fa7760
  Stored in directory: /root/.cache/pip/wheels/ee/ee/18/c466221ca6900e3efce2f4ea9c329288808679aecdcb2838d3
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=c6217d45cdcc78334097a074a8c07fe85e88eb767e8b6276b6da7ba3fb7e40b2
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=dc61208a8d85150f220820e4844561345f6a12a7c790bff9ac63

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


In [4]:
#@title Re-Instalar el Object Detection API
# Nota: se ejecuta de nuevo para que lo instale bien
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1644507 sha256=91a98abd8bcc4c7d3d441ff67a200079651ace0286494080aecc4ea7737c1b56
  Stored in directory: /tmp/pip-ephem-wheel-cache-h3bcx_4n/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


1) Define las librerías a utilizar:

In [5]:
#@title Cargar Librerías
import os
import glob
import pandas as pd

import xml.etree.ElementTree as ET

import io
import tensorflow as tf

from PIL import Image
from tqdm import tqdm
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

print("Librerías cargadas.")

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Librerías cargadas.


2) Monta el Drive y define archivos a utilizar:

In [6]:
#@title Montar Drive y definir archivos a procesar

# monta Google Drive nuevamente (se pierde conexión cuando se reinicia el entrorno anterior)
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# configuración de directorios local en Google Drive

model_data_dir_path = '/content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles' #@param {type:"string"}
data_path_Train = '/content/gdrive/MyDrive/IA/demoObjDet-Carteles/Carteles/train' #@param {type:"string"}
data_path_Test = '/content/gdrive/MyDrive/IA/demoObjDet-Carteles/Carteles/validation' #@param {type:"string"}


# define las carpetas de XMLs
train_image_dir = data_path_Train + '/images'
train_xml_dir = data_path_Train + '/annotations'
test_image_dir =data_path_Test + '/images'
test_xml_dir = data_path_Test + '/annotations' 
print("> train_image_dir: ", train_image_dir)
print("> train_xml_dir: ", train_xml_dir)
print("> test_image_dir: ", test_image_dir)
print("> test_xml_dir: ", test_xml_dir)
print("\n")

# define los nombres de los archivos
train_csv_fname = model_data_dir_path + '/train_labels.csv'
test_csv_fname = model_data_dir_path + '/test_labels.csv'
print("> train_csv_fname: ", train_csv_fname)
print("> test_csv_fname: ", test_csv_fname)

label_map_pbtxt_fname = model_data_dir_path + '/label_map.pbtxt'
print("> label_map_pbtxt_fname: ", label_map_pbtxt_fname)

train_record_fname = model_data_dir_path + '/train.record'
test_record_fname = model_data_dir_path + '/test.record'
print("> train_record_fname: ", train_record_fname)
print("> test_record_fname: ", test_record_fname)

Mounted at /content/gdrive
> train_image_dir:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles/Carteles/train/images
> train_xml_dir:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles/Carteles/train/annotations
> test_image_dir:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles/Carteles/validation/images
> test_xml_dir:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles/Carteles/validation/annotations


> train_csv_fname:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/train_labels.csv
> test_csv_fname:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/test_labels.csv
> label_map_pbtxt_fname:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/label_map.pbtxt
> train_record_fname:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/train.record
> test_record_fname:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/test.record


3) Genera CSVs auxiliares en base a los XMLs correspondientes a las imágenes: 

In [7]:
#@title Definir funciones auxiliares para CSV
def __list_to_csv(annotations, output_file):
    column_name = [
        'filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'
    ]
    xml_df = pd.DataFrame(annotations, columns=column_name)
    xml_df.to_csv(output_file, index=None)

def xml_to_csv(xml_dir, output_file):
    """Reads all XML files, generated by labelImg, from a directory and generates a single CSV file"""
    annotations = []
    xmlFileList = glob.glob(xml_dir + '/*.xml')
    total = len(xmlFileList)
    auxMostrarTotal = (total//20)
    cant = 0
    print("XMLs a procesar en ", xml_dir, ": ", total)
    for xml_file in xmlFileList:
        cant = cant + 1
        if (cant % auxMostrarTotal) == 0:
          print("\t\t ya se procesaron  ", cant, " xmls")
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text), member[0].text,
                     int(member[4][0].text), int(member[4][1].text),
                     int(member[4][2].text), int(member[4][3].text))
            annotations.append(value)

    __list_to_csv(annotations, output_file)
    print(output_file, " generado. ")

print("Funciones auxiliares para CSV definidas")    

Funciones auxiliares para CSV definidas


In [8]:
#@title Generar CSVs

# Por las dudas, se elimina los archivos si ya existen (para asegurar su actualización)
if os.path.isfile(train_csv_fname):
  os.remove(train_csv_fname)
  print(train_csv_fname, " eliminado.")

if os.path.isfile(test_csv_fname):
  os.remove(test_csv_fname)
  print(test_csv_fname, " eliminado.")

# genera CSVs
print("\n")
print("-- generando train_csv: ", train_csv_fname)
xml_to_csv(train_xml_dir, train_csv_fname)
print("> train_csv generado.")
print("\n")
print("-- generando test_csv: ", test_csv_fname)
xml_to_csv(test_xml_dir, test_csv_fname)
print("> test_csv generado.")

/content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/train_labels.csv  eliminado.
/content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/test_labels.csv  eliminado.


-- generando train_csv:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/train_labels.csv
XMLs a procesar en  /content/gdrive/MyDrive/IA/demoObjDet-Carteles/Carteles/train/annotations :  1751
		 ya se procesaron   87  xmls
		 ya se procesaron   174  xmls
		 ya se procesaron   261  xmls
		 ya se procesaron   348  xmls
		 ya se procesaron   435  xmls
/content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/train_labels.csv  generado. 
> train_csv generado.


-- generando test_csv:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/test_labels.csv
XMLs a procesar en  /content/gdrive/MyDrive/IA/demoObjDet-Carteles/Carteles/validation/annotations :  602
		 ya se procesaron   30  xmls
		 ya se procesaron   60  xmls
		 ya se procesaron   90  xmls
		 ya se procesaron   120  xmls
		 ya se proce

4) Genera archivo de clases label_map_pbtxt:

In [9]:
#@title Definir funciones auxiliares para label_map
def pbtxt_from_csv(csv_path, pbtxt_path):
    class_list = list(pd.read_csv(csv_path)['class'].unique())
    class_list.sort()

    pbtxt_from_classlist(class_list, pbtxt_path)


def pbtxt_from_classlist(l, pbtxt_path):
    pbtxt_text = ''

    for i, c in enumerate(l):
        pbtxt_text += 'item {\n    id: ' + str(
            i + 1) + '\n    display_name: "' + c + '"\n}\n\n'

    with open(pbtxt_path, "w+") as pbtxt_file:
        pbtxt_file.write(pbtxt_text)

print("Funciones auxiliares para label_map definidas")    

Funciones auxiliares para label_map definidas


In [10]:
#@title Generar label_map_pbtx

#Por las dudas, se elimina los archivos si ya existen (para asegurar su actualización)
if os.path.isfile(label_map_pbtxt_fname):
  os.remove(label_map_pbtxt_fname)
  print(label_map_pbtxt_fname, " eliminado.")

print("\n")
print("-- generando label_map_pbtxt: ", label_map_pbtxt_fname)
pbtxt_from_csv(train_csv_fname, label_map_pbtxt_fname)
print("> label_map_pbtxt generado.")

/content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/label_map.pbtxt  eliminado.


-- generando label_map_pbtxt:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/label_map.pbtxt
> label_map_pbtxt generado.


5) Generar TFRecords en base a los XMLs correspondientes a las imágenes:

In [11]:
#@title Definir funciones auxiliares para TFRecords
def generate_TFrecords(pbtxt_input, csv_input, image_dir, output_path):
    class_dict = class_dict_from_pbtxt(pbtxt_input)

    writer = tf.io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = pd.read_csv(csv_input)
    grouped = __split(examples, 'filename')

    for group in tqdm(grouped, desc='groups'):
        tf_example = create_tf_example(group, path, class_dict)
        writer.write(tf_example.SerializeToString())

    writer.close()

def create_tf_example(group, path, class_dict):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)),
                        'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        if set(['xmin_rel', 'xmax_rel', 'ymin_rel', 'ymax_rel']).issubset(
                set(row.index)):
            xmin = row['xmin_rel']
            xmax = row['xmax_rel']
            ymin = row['ymin_rel']
            ymax = row['ymax_rel']

        elif set(['xmin', 'xmax', 'ymin', 'ymax']).issubset(set(row.index)):
            xmin = row['xmin'] / width
            xmax = row['xmax'] / width
            ymin = row['ymin'] / height
            ymax = row['ymax'] / height

        xmins.append(xmin)
        xmaxs.append(xmax)
        ymins.append(ymin)
        ymaxs.append(ymax)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_dict[row['class']])

    tf_example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'image/height':
                dataset_util.int64_feature(height),
                'image/width':
                dataset_util.int64_feature(width),
                'image/filename':
                dataset_util.bytes_feature(filename),
                'image/source_id':
                dataset_util.bytes_feature(filename),
                'image/encoded':
                dataset_util.bytes_feature(encoded_jpg),
                'image/format':
                dataset_util.bytes_feature(image_format),
                'image/object/bbox/xmin':
                dataset_util.float_list_feature(xmins),
                'image/object/bbox/xmax':
                dataset_util.float_list_feature(xmaxs),
                'image/object/bbox/ymin':
                dataset_util.float_list_feature(ymins),
                'image/object/bbox/ymax':
                dataset_util.float_list_feature(ymaxs),
                'image/object/class/text':
                dataset_util.bytes_list_feature(classes_text),
                'image/object/class/label':
                dataset_util.int64_list_feature(classes),
            }))
    return tf_example

def __split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [
        data(filename, gb.get_group(x))
        for filename, x in zip(gb.groups.keys(), gb.groups)
    ]

def class_dict_from_pbtxt(pbtxt_path):
    # open file, strip \n, trim lines and keep only
    # lines beginning with id or display_name
    data = [
        l.rstrip('\n').strip()
        for l in open(pbtxt_path, 'r', encoding='utf-8-sig')
        if 'id:' in l or 'display_name:'
    ]
    ids = [int(l.replace('id:', '')) for l in data if l.startswith('id')]
    names = [
        l.replace('display_name:', '').replace('"', '').strip() for l in data
        if l.startswith('display_name')
    ]

    #print(data)

    # join ids and display_names into a single dictionary
    class_dict = {}
    for i in range(len(ids)):
        class_dict[names[i]] = ids[i]

    return class_dict


print("Funciones auxiliares para TFRecords definidas")   

Funciones auxiliares para TFRecords definidas


In [12]:
#@title Generar TFRecords

# por las dudas, se elimina los archivos si ya existen (para asegurar su actualización)
if os.path.isfile(train_record_fname):
  os.remove(train_record_fname)
  print(train_record_fname, " eliminado.")

if os.path.isfile(test_record_fname):
  os.remove(test_record_fname)
  print(test_record_fname, " eliminado.")

print("\n")
print("-- generando train_TFRecords: ", train_record_fname)
generate_TFrecords(label_map_pbtxt_fname, train_csv_fname, train_image_dir, train_record_fname)
print("\n> train_TFRecords generado.")
print("\n")
print("-- generando test_TFRecords: ", test_record_fname)
generate_TFrecords(label_map_pbtxt_fname, test_csv_fname, test_image_dir, test_record_fname)
print("\n> test_TFRecords generado.")

/content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/train.record  eliminado.
/content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/test.record  eliminado.


-- generando train_TFRecords:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/train.record


groups:   0%|          | 0/499 [00:00<?, ?it/s]


> train_TFRecords generado.


-- generando test_TFRecords:  /content/gdrive/MyDrive/IA/demoObjDet-Carteles-TF2/Carteles/test.record


groups: 100%|██████████| 499/499 [02:12<00:00,  3.77it/s]


> test_TFRecords generado.
